# Исследование надежности заемщиков.

## Импорт библиотек, загрузка и обзор данных

In [1]:
import pandas as pd

In [2]:

data=pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


- Есть пропущенные значения в колонках days_empoyed и total_income. Предположительно, в других колонках пропусков нет, но это подтвердится/опровергнется после подсчёта числа пропусков в одном из столбцов. 

 - В total_income тип данных - float. Если нас интересует разбиение по типам дохода, то дробной частью можно пренебречь.

- Аналогично - в days_empoyed: важно число полных отработанных дней, а не доли дня в часах.

## Предобработка данных

### Заполнение пропусков

In [3]:
#Посмотрим, сколько всего пропусков в колонках. Предположительно, 2174 - если в остальных столбцах нет пропусков
len(data[data['days_employed'].isna()])

2174

Гипотеза подтвердилась.Итак, пропущено 2174/21525 - примерно 10% значений. Вероятно, это может быть связано как с человеческим фактором (ошибка при вводе/отказ клиента предоставить информацию), так и с ошибками при переносе данных. 10% - существенная часть данных, и потому просто удалить нельзя. Оптимальной будет замена пропусков медианным значением: так искажение данных будет минимальным.(Cреднее не подойдёт, т.к при наличии сильно выделяющегося значения оно "перетянет одеяло на себя". Как среднее по палате: если у 4 пациентов 36.6, а у одного - 40.0- средняя температура = 37.2. Объективные выводы из этого сделать сложно :) )

In [4]:
median_days_employed = data['days_employed'].median()
# Медиана отрицательная?  Интересно...
data['days_employed']=data['days_employed'].abs() #Удалять половину данных не хочется: рискнём и просто возьмём модуль
data['days_employed'].sort_values() #Проверим, получилось ли
median_days_employed = int(data['days_employed'].median())
median_income=data['total_income'].median()

data['days_employed']=data['days_employed'].fillna(median_days_employed) #Меняем пропушенные значения на медианное
data['total_income']=data['total_income'].fillna(median_income) #И ещё раз для другого столбца

### Проверка данных на аномалии и исправления.

In [5]:
data.sort_values(by='days_employed').head() 
data.sort_values(by='days_employed').tail(3446)

#Величина "хвоста" определена эмпирически: проверялось, с какого момента теряется адекватность

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
20444,0,328728.720605,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,96519.339647,покупка жилья для семьи
9328,2,328734.923996,41,высшее,0,женат / замужем,0,M,пенсионер,0,126997.497760,операции со своей недвижимостью
17782,0,328771.341387,56,среднее,1,женат / замужем,0,F,пенсионер,0,68648.047062,операции с коммерческой недвижимостью
14783,0,328795.726728,62,высшее,0,женат / замужем,0,F,пенсионер,0,79940.196752,на покупку своего автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования


Последние 3445 чисел дней трудового стажа слишком огромные. Получается, трудовой стаж некоторых клиентов превышает 900 лет. 
Возможно, записали число часов вместо дней; а возможно, ошиблись с расположением точки - и, следовательно, с порядком числа.
Предположу, что дело в первом варианте: для второго -то есть, для простой опечатки -  таких ошибок слишком много, так что логичнее выглядит версия о том, что кто-то перепутал и занёс число рабочих часов.

//Как он это считал, бедный...

Проверим остальные столбцы. Аномально высокие значения могут быть в столбцах dob_years и children. Количественной переменной является и total_income, но верхнюю границу "адекватных результатов" провести сложно. Проверим, нет ли отрицательных значений

In [6]:
data.sort_values(by='dob_years').head(10) #Отрицательных возрастов нет, хотя опыт работы в 0 лет настораживает
data.sort_values(by='dob_years').tail(10) 
#Максимальный возраст - 75. Интересно, что цель кредита "обучение", но признать неадекваным не можем: теоретически это возможно

data.sort_values(by='children').head(48) 
#У 47 людей -1 ребёнок. Либо это был прочерк, либо "минус" лишний. Большая часть пар семейная, так что предположим 2-ой вариант
data.sort_values(by='children').tail(77)
#У 76 людей -20 детей. Можно было бы предположить, что настолько многодетные семьи - тоже клиенты банка
# Но тогда были бы и 19, 18, 17 детей..Предположу, что "0" лишний, и имелось в виду 2

data.sort_values(by='total_income').head() #Отрицательных значений нет - считаем столбец достоверным

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14585,0,359219.059341,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667.263793,недвижимость
13006,0,369708.589113,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205.280566,заняться высшим образованием
16174,1,3642.820023,52,Среднее,1,женат / замужем,0,M,сотрудник,0,21367.648356,приобретение автомобиля
1598,0,359726.104207,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695.101789,на проведение свадьбы
14276,0,346602.453782,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895.614355,недвижимость


Теперь проверим категориальные переменные на отсутствие неожиданных значений. Оно пригодится ещё при удалении дубликатов, но вдруг кто-то умудрился учиться в Хогвартсе  и ощущает себя боевым вертолётом?

In [7]:
data['education'].unique()
data['education_id'].unique()
data['family_status'].unique()
data['family_status_id'].unique()
data['gender'].unique() #XNA? Интересно... Вызовем строчки с ним
data[data['gender']=='XNA'] #Он такой один, так что оставим. Но непонятно.
data['income_type'].unique()
data['debt'].unique()
data['purpose'].unique()


array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Итак, аномальные значения найдены в столбцах 'children' - отрицательное количество детей и 20 детей; 'days_employed' - больше 900 лет рабочего стажа. Также обнаружены дубликаты в столбцах 'education' и 'purpose', и странное значение 'XNA' в гендере одного из клиентов (неожиданно угадала с боевым вертолётом?) . XNA оставили, с дубликатами разберёмся позднее.
Преобразуем аномальные значения в соответствии с нашими предположениями

In [8]:
data['children']=data['children'].abs() #Теперь отрицательного количества детей нет, только 0 и больше
data['children']=data['children'].replace(20,2) #В соответствии с предположением, что 20 детей означало 2, заменим на нужное число

data['dob_years']=data['dob_years'].replace(0,15) #поменяем возраст на адекватный: сделаем минимальный возраст=15
data['dob_years'].min() #Теперь минимальное значение=15

data.sort_values(by='days_employed').tail(3446)

def max_stage(row): 
#Функция делит стаж, больший максимально возможного, на 24 (т.к предполагаем, что это число часов - переводим в дни), и оставляет адекватный
    current_age = row['days_employed']
    max_age= 21900 #60*365 - максимальный стаж, выраженный в днях
    if current_age > max_age:
        return (current_age/24)
    else:
        return current_age
data['days_employed'] = data.apply(max_stage,axis=1)    
data.sort_values(by='days_employed').tail(3446)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14856,0,13724.473816,60,среднее,1,гражданский брак,1,F,пенсионер,0,124468.944722,образование
8354,0,13725.427022,54,СРЕДНЕЕ,1,гражданский брак,1,F,пенсионер,0,95950.360328,сыграть свадьбу
9166,0,13725.510457,68,среднее,1,женат / замужем,0,F,пенсионер,0,101396.295670,заняться образованием
11157,0,13725.703094,71,СРЕДНЕЕ,1,вдовец / вдова,2,F,пенсионер,0,39406.483691,покупка жилой недвижимости
16253,0,13727.539247,53,среднее,1,гражданский брак,1,F,пенсионер,0,94973.587101,заняться высшим образованием
...,...,...,...,...,...,...,...,...,...,...,...,...
7664,1,16736.462226,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
10006,0,16738.158823,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
6954,0,16739.808353,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
4299,0,17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья


###  Изменение типов данных.

In [9]:
data.info() #Вспомним, что не тот тип данных у days_employed  и total_income. Поменяем на int
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------     

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


Теперь все данные имеют нужный тип. 

### Удаление дубликатов.

Мы уже проверяли данные на уникальность, когда искали аномальные значения.  Были обнаружены дубликаты в столбцах 'education' и 'purpose'.

In [10]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [11]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [12]:
data["education"] = data["education"].str.lower()
data["education"].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

В столбце purpose заменой букв на строчные не обойдёмся: регистр тут одинаковый, а вот формулировки разные...

In [13]:
data['purpose']=data['purpose'].replace('сыграть свадьбу','свадьба').replace('на проведение свадьбы','свадьба')
data['purpose']=data['purpose'].replace('дополнительное образование','образование').replace('заняться образованием','образование').replace('получение дополнительного образования','образование').replace('заняться образованием','образование').replace('заняться высшим образованием','образование').replace('профильное образование','образование').replace('получение высшего образования','образование').replace('высшее образование','образование').replace('получение образования','образование')
data['purpose']=data['purpose'].replace('операции с жильем', 'недвижимость').replace('покупка жилья для семьи', 'недвижимость').replace('покупка недвижимости', 'недвижимость').replace('покупка коммерческой недвижимости', 'недвижимость').replace('покупка жилой недвижимости', 'недвижимость').replace('строительство собственной недвижимости', 'недвижимость').replace('строительство недвижимости', 'недвижимость').replace('операции со своей недвижимостью', 'недвижимость').replace('операции с коммерческой недвижимостью', 'недвижимость')
data['purpose']=data['purpose'].replace('покупка жилья', 'недвижимость').replace('строительство жилой недвижимости', 'недвижимость').replace('покупка жилья для сдачи', 'недвижимость').replace('покупка своего жилья', 'недвижимость').replace('ремонт жилью', 'недвижимость').replace('жилье', 'недвижимость').replace('операции с недвижимостью', 'недвижимость')
data['purpose']=data['purpose'].replace('приобретение автомобиля', 'автомобиль').replace('на покупку подержанного автомобиля', 'автомобиль').replace('на покупку своего автомобиля', 'автомобиль').replace('автомобили', 'автомобиль').replace('сделка с подержанным автомобилем', 'автомобиль').replace('сделка с автомобилем', 'автомобиль').replace('свой автомобиль', 'автомобиль').replace('на покупку автомобиля', 'автомобиль')
data['purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

Подобные дубликаты могли появиться из-за человеческого фактора: разные люди по-разному формулируют одну и ту же причину (и выбирают регистр)
Проверим таблицу на дубликаты строк (что означало бы, что один и тот же клиент был внесен в БД несколько раз - может быть как технической, так и человеческой ошибкой).

In [14]:
data.duplicated().sum() #405 - не критично, но и немало
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum() #Все дубликаты удалены
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21115,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,недвижимость
21116,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль
21117,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21118,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль


###  Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Cоздадим дополнительные датафреймы: с образованием и его идентификатором и, аналогично, с семейным статусом

In [15]:
education_dict=data[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop = True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Нелогичное распределение id (логичнее было 0 - начальное, 2- неоконченное высшее, 3 - высшее), нуданесуть 

In [16]:
family_dict=data[['family_status_id','family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop = True)
family_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [17]:
#Удалим лишние столбцы из исходной таблицы
data=data.drop('education',1)
data=data.drop('family_status',1)
data


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,1,0,M,сотрудник,0,267628,образование
4,0,14177,53,1,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...
21115,1,4529,43,1,1,F,компаньон,0,224791,недвижимость
21116,0,14330,67,1,0,F,пенсионер,0,155999,автомобиль
21117,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость
21118,3,3112,38,1,0,M,сотрудник,1,244093,автомобиль


## Категоризация

###  Категоризация дохода.

Создадим столбец total_income_category с категориями дохода

In [18]:
def income_category(row): #Функция, определяющая категорию дохода
    income = row['total_income']
    if income <=30000:
        category='E'    
    elif income <=50000:
        category='D'    
    elif income <=200000:
        category='C'  
    elif income <=1000000:
        category='B'    
    else:
        category='A'    
    return category
data['income_category']=data.apply(income_category, axis=1)
data.sort_values(by='total_income').head(10)
data.sort_values(by='total_income').tail(10)
#Выглядит правильно. В отсортированной по доходу таблице в начале категория "Е", в конце "А"

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_category
10945,1,1851,36,0,1,F,сотрудник,0,1286280,недвижимость,A
15050,1,10207,64,0,3,M,компаньон,0,1350245,недвижимость,A
18055,1,3173,41,0,4,F,компаньон,0,1427934,автомобиль,A
18070,1,333,41,0,1,M,компаньон,0,1551152,свадьба,A
17229,0,2285,43,1,0,M,компаньон,0,1597613,недвижимость,A
16914,0,5734,42,0,1,M,компаньон,0,1711309,свадьба,A
20423,0,4719,61,1,4,F,сотрудник,0,1715018,недвижимость,A
9082,1,5248,35,1,1,M,сотрудник,0,1726276,образование,A
19265,1,2577,39,0,0,M,компаньон,1,2200852,недвижимость,A
12262,0,1477,44,0,0,M,компаньон,0,2265604,недвижимость,A


###  Категоризация целей кредита.

In [19]:
#Создадим таблицу, демонстрирующую зависимость между количеством детей и наличием долга
debt_and_children = data.groupby(['children','debt']).agg({'debt':["count"]})
debt_and_children

debt
               count
children debt       
0        0     12768
         1      1061
1        0      4353
         1       445
2        0      1913
         1       202
3        0       302
         1        27
4        0        36
         1         4
5        0         9

Таблица недостаточно наглядна,хотя в ней уже есть необходимые данные. Но интереснее сразу посмотреть на процент должников от общего числа клиентов

## Необходимые расчёты

In [20]:
debtors=data.groupby('children')['debt'].sum()
clients=data.groupby('children')['debt'].count()
percentage=debtors*100/clients
percentage


children
0     7.672283
1     9.274698
2     9.550827
3     8.206687
4    10.000000
5     0.000000
Name: debt, dtype: float64

Итак, с увеличением числа детей вероятность не отдать долг растёт. Выпадает только результат для клиентов с 5 детьми. Это объяснимо малой выборкой: таких клиентов было всего 9.

Аналогично для других пунктов

In [21]:
debtors=data.groupby('family_status_id')['debt'].sum()
clients=data.groupby('family_status_id')['debt'].count()
percentage=debtors*100/clients
percentage

family_status_id
0    7.692945
1    9.408341
2    6.680806
3    7.124895
4    9.841954
Name: debt, dtype: float64

In [22]:
debtors=data.groupby('income_category')['debt'].sum()
clients=data.groupby('income_category')['debt'].count()
percentage=debtors*100/clients
percentage

income_category
A    8.000000
B    7.062091
C    8.659610
D    6.000000
E    9.090909
Name: debt, dtype: float64

In [23]:
debtors=data.groupby('purpose')['debt'].sum()
clients=data.groupby('purpose')['debt'].count()
percentage=debtors*100/clients
percentage

purpose
автомобиль      9.410112
недвижимость    7.383248
образование     9.334006
свадьба         8.065915
Name: debt, dtype: float64

## Ответы на вопросы.

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Да. С увеличением числа детей вырастает вероятность, что человек не вернёт кредит в срок

### Есть ли зависимость между семейным положением и возвратом кредита в срок?
Чаще всего должниками становятся люди, состоящие в гражданском браке (9,4%) и неженатые (незамужние). Реже всего - вдовствующие

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Как ни странно, прямой зависимости нет. Реже всего кредит возвращают вовремя люди с самым низким доходом (что логично), но далее зависимость идёт нелинейно.  Чаще всего вовремя отдают люди с доходом категории D.

### Как разные цели кредита влияют на его возврат в срок?
Чаще всего вовремя отдают кредиты, взятые на операции с недвижимостью. Реже всего -  на образование и операции с автомобилями

### Вывод 
Наиболее безопасными для банка заёмщиками являются вдовствующие клиенты без детей со средним доходом, берущие кредит на операции с недвижимостью

## Общий вывод:

<strike>Оторвать руки тому</strike> Отправить в срочный отпуск человека, который составлял эту таблицу. Большая часть данных так или иначе повреждена, и восстановлена исключительно из моих предположений, что там могло быть. Я бы не рекомендовала делать сколько-нибудь серьёзные выводы по этим результатам и основывать на них какие бы то ни было решения.

Выводы, которые можно сделать на основании этих данных:
### Влияние количества детей:
По предоставленным данным, невовремя вернули кредит: 7.7% клиентов без детей; 9.3% - с одним ребёнком; 9.6% - с двумя; 8.2%  - с тремя; 10% - с четырьмя. Все клиенты с 5 детьми вернули кредит вовремя, однако для них выборка крайне мала: 9 человек.  
Вывод: С увеличением числа детей вырастает вероятность, что клиент не вернёт кредит в срок

### Влияние семейного положения:
По предоставленным данным, невовремя вернули кредит:7.7% клиентов, состоящих в браке; 9.4% клиентов в гражданском браке; 6.7% вдовствующих клиентов; 7.1% клиентов в разводе; 9.8% не женатых/не замужних.  
Вывод: Чаще всего должниками становятся люди, состоящие в гражданском браке и неженатые (незамужние). Реже всего - вдовствующие

###  Влияние уровней дохода: 
По предоставленным данным, невовремя вернули кредит: 8% клиентов с категорией доходов А; 7% - с категорией дохода B; 8.7% - с категорией дохода C; 6% - с категорией дохода D; 9% - с категорией дохода E.  
Вывод: прямой зависимости нет. Реже всего кредит возвращают вовремя люди с самым низким доходом, но далее зависимость идёт нелинейно. Чаще всего вовремя отдают люди с доходом категории D.

### Влияние целей кредита:
По предоставленным данным, невовремя вернули кредит: 9.4% клиентов, взявших его на операции с автомобилем; 7.4% - на операции с недвижимостью; 9.3% - на образование; 8.1% - на свадьбу. 
Вывод: Чаще всего вовремя отдают кредиты, взятые на операции с недвижимостью. Реже всего - на образование и операции с автомобилями